In [59]:
import osmnx as ox
import networkx as nx
import rustworkx as rx
import numpy as np
import h3
from timeit import default_timer as timer

timetable = {}

def node_to_hex(node, resolution):
    lat = node['y']
    lon = node['x']
    return h3.latlng_to_cell(lat, lon, resolution)

def nodes_to_hexes(G, resolution):
    """Maps H3 hexagons to their contained nodes."""
    nodes = G.nodes(data=True)
    h3_to_nodes = {}
    
    for node_id, data in nodes:
        h3_hex = node_to_hex(data, resolution)
        if h3_hex not in h3_to_nodes:
            h3_to_nodes[h3_hex] = []
        h3_to_nodes[h3_hex].append(node_id)
    
    return h3_to_nodes

def create_hex_pivot(node_mapping: np.ndarray) -> np.ndarray:
    """
    Creates a structured array of unique hexes with their contained node IDs.
    
    Returns a numpy array with dtype:
        - hex_id: The H3 hex identifier
        - nx_ids: Array of NetworkX node IDs in this hex
        - rx_idxs: Array of RustWorkX indices in this hex
    """
    # Get unique hex IDs and their counts
    unique_hexes, hex_counts = np.unique(node_mapping['hex_id'], return_counts=True)
    
    # Create dtype for the pivoted array
    pivot_dtype = np.dtype([
        ('hex_id', object),
        ('nx_ids', object),  # Will store arrays of varying length
        ('rx_idxs', object)  # Will store arrays of varying length
    ])
    
    # Initialize the pivoted array
    hex_pivot = np.empty(len(unique_hexes), dtype=pivot_dtype)
    
    # Fill the pivoted array
    for i, hex_id in enumerate(unique_hexes):
        # Get all nodes in this hex
        hex_mask = node_mapping['hex_id'] == hex_id
        nodes_in_hex = node_mapping[hex_mask]
        
        hex_pivot[i] = (
            hex_id,
            nodes_in_hex['nx_id'],  # nx_ids
            nodes_in_hex['rx_idx']    # rx_idxs
        )
    
    return hex_pivot

def retrieve_node_mapping_by_rx_idx(node_mapping, rx_idx):
    return node_mapping[np.where(node_mapping['rx_idx'] == rx_idx)][0]

def retrieve_node_mapping_by_nx_id(node_mapping, nx_id):
    return node_mapping[np.where(node_mapping['nx_id'] == nx_id)][0]

def rx_to_nx_node(rx_idx):
    return node_mapping[np.where(node_mapping['rx_idx'] == rx_idx)][0][0]

def nx_node_to_rx(nx_id):
    return node_mapping[np.where(node_mapping['nx_id'] == nx_id)][0][4]

# Step 1: Download Street Network
start = timer()
place_name = "Brenham, Texas, USA"
G = ox.graph_from_place(place_name, network_type='drive')
end = timer()
download_time = end - start
timetable['download_time'] = download_time
print(f"Street Network Download: {download_time:.2f} seconds")

# Step 2: Create node mapping with H3 indices
start = timer()

# Define the dtype for our structured array
node_dtype = np.dtype([
    ('nx_id', object),    # NetworkX node ID
    ('lon', np.float64),    # longitude
    ('lat', np.float64),    # latitude
    ('hex_id', object),     # H3 hex ID
    ('rx_idx', np.int64)    # Rustworkx index
])

# Pre-allocate the array based on number of nodes
G_rx = rx.PyGraph()
num_nodes = len(G)
node_mapping = np.empty(num_nodes, dtype=node_dtype)

# Fill the array with node information
for i, (node, data) in enumerate(G.nodes(data=True)):
    lat = data.get('y', 0.0)
    lon = data.get('x', 0.0)
    hex_id = h3.latlng_to_cell(lat, lon, 9)
    rx_idx = G_rx.add_node(i)  # Store the node data if needed
    
    node_mapping[i] = (
        node,       # node_id
        lon,        # longitude
        lat,        # latitude
        hex_id,     # H3 hex ID
        rx_idx      # Rustworkx index
    )

# Create edge connections
for u, v, data in G.edges(data=True):
    # Use searchsorted for efficient index lookup
    u_mask = node_mapping['nx_id'] == u
    v_mask = node_mapping['nx_id'] == v
    u_idx = node_mapping['rx_idx'][u_mask][0]
    v_idx = node_mapping['rx_idx'][v_mask][0]
    weight = data.get('length', 1.0)
    G_rx.add_edge(u_idx, v_idx, weight)

end = timer()
mapping_time = end - start
timetable['mapping_time'] = mapping_time
print(f"Node Mapping Creation: {mapping_time:.2f} seconds")
print(f"NetworkX Graph Contains {len(G.nodes())} nodes and {len(G.edges())} edges")
print(f"RustworkX Graph Contains {len(G_rx.nodes())} nodes and {len(G_rx.edges())} edges")

# Step 3: Create Distance Matrix between Nodes
start = timer()
# pre-computed matrix containing all shortest path distances between nodes
dist_matrix = rx.floyd_warshall_numpy(G_rx, weight_fn=lambda x: float(x))
np.fill_diagonal(dist_matrix, np.inf)
distance_calc = timer() - start
timetable['distance_calc'] = distance_calc
print(f"Node Distance Matrix Creation: {distance_calc:.2f} seconds")

# Step 4: Create Distance Matrix between Hexagons
start = timer()
h3_to_anchor_node = {hex_id: nodes[0] for hex_id, nodes in nodes_to_hexes(G, 8).items()}

hex_pairs = {}
for hex1, anchor1 in h3_to_anchor_node.items():
    for hex2, anchor2 in h3_to_anchor_node.items():
        idx1 = nx_node_to_rx(anchor1)
        idx2 = nx_node_to_rx(anchor2)
        dist = dist_matrix[idx1, idx2]
        if not np.isinf(dist):
            hex_pairs[(hex1, hex2)] = dist
hex_pairs_calc = timer() - start
timetable['hex_pairs_calc'] = hex_pairs_calc
print(f"Hex Pairs Calculation: {hex_pairs_calc:.2f} seconds")
print(f"Graph Contains {len(np.unique(node_mapping['hex_id']))} unique hexagons")

def get_or_create_anchor_node(hex_id):
    if hex_id not in h3_to_anchor_node:
        # Fallback: Pick any node in the hex if it wasn't precomputed
        nodes_in_hex = node_mapping[node_mapping['hex_id'] == hex_id]
        if len(nodes_in_hex) == 0:
            raise ValueError(f"No nodes found in hex {hex_id}")
        h3_to_anchor_node[hex_id] = nodes_in_hex['nx_id'][0]
    return h3_to_anchor_node[hex_id]

def find_shortest_path(origin_node, destination_node):
    origin_hex = node_to_hex(G.nodes[origin_node], 8)
    destination_hex = node_to_hex(G.nodes[destination_node], 8)

    # Try origin fallback if anchor is missing
    if origin_hex not in h3_to_anchor_node:
        origin_hex = find_nearest_valid_hex(origin_hex, h3_to_anchor_node)
        if origin_hex is None:
            print(f"Skipping: No valid anchor for origin {origin_node} in hex {origin_hex}")
            return None

    # Try destination fallback if anchor is missing
    if destination_hex not in h3_to_anchor_node:
        destination_hex = find_nearest_valid_hex(destination_hex, h3_to_anchor_node)
        if destination_hex is None:
            print(f"Skipping: No valid anchor for destination {destination_node} in hex {destination_hex}")
            return None

    anchor_origin = h3_to_anchor_node[origin_hex]
    anchor_dest = h3_to_anchor_node[destination_hex]

    origin_rx = nx_node_to_rx(origin_node)
    anchor_origin_rx = nx_node_to_rx(anchor_origin)
    anchor_dest_rx = nx_node_to_rx(anchor_dest)
    destination_rx = nx_node_to_rx(destination_node)

    same_anchor_hex = origin_hex == destination_hex
    same_origin_anchor = origin_node == anchor_origin
    same_dest_anchor = destination_node == anchor_dest

    # 1. Origin -> Anchor Origin
    if same_origin_anchor:
        path_nx_1 = [origin_node]
    else:
        path_rx_1 = rx.dijkstra_shortest_paths(G_rx, origin_rx, weight_fn=lambda x: float(x), target=anchor_origin_rx)
        if anchor_origin_rx not in path_rx_1:
            print(f"Skipping trip: No path from origin {origin_node} to anchor_origin {anchor_origin}")
            return None
        path_nx_1 = [rx_to_nx_node(i) for i in path_rx_1[anchor_origin_rx]]

    # 2. Anchor Origin -> Anchor Dest (Precomputed or Same)
    if same_anchor_hex:
        path_nx_2 = []  # They're the same; this segment is empty
    else:
        if (origin_hex, destination_hex) in hex_pairs:
            path_nx_2 = [anchor_origin, anchor_dest]  # Placeholder (precomputed distance used)
        else:
            path_rx_2 = rx.dijkstra_shortest_paths(G_rx, anchor_origin_rx, lambda x: float(x), target=anchor_dest_rx)
            if anchor_dest_rx not in path_rx_2:
                print(f"Skipping trip: No path between anchor_origin {anchor_origin} and anchor_dest {anchor_dest}")
                return None
            path_nx_2 = [rx_to_nx_node(i) for i in path_rx_2[anchor_dest_rx]]

    # 3. Anchor Dest -> Destination
    if same_dest_anchor:
        path_nx_3 = [destination_node]
    else:
        path_rx_3 = rx.dijkstra_shortest_paths(G_rx, anchor_dest_rx, weight_fn=lambda x: float(x), target=destination_rx)
        if destination_rx not in path_rx_3:
            print(f"Skipping trip: No path from anchor_dest {anchor_dest} to destination {destination_node}")
            return None
        path_nx_3 = [rx_to_nx_node(i) for i in path_rx_3[destination_rx]]

    # Combine the paths
    full_path_nx = path_nx_1 + path_nx_2[1:] + path_nx_3[1:]
    return full_path_nx

source_test_nodes = np.random.choice(G.nodes(), 100)
target_test_nodes = np.random.choice(G.nodes(), 100)
test_trips = list(zip(source_test_nodes, target_test_nodes))

execution_times = []
for i, trip in enumerate(test_trips):
    start = timer()
    find_shortest_path(trip[0], trip[1])
    end = timer()
    execution_times.append(end - start)

print(f"Average Execution Time: {np.mean(execution_times):.6f} seconds per trip")

Street Network Download: 0.41 seconds
Node Mapping Creation: 0.05 seconds
NetworkX Graph Contains 1154 nodes and 3020 edges
RustworkX Graph Contains 1154 nodes and 3020 edges
Node Distance Matrix Creation: 0.22 seconds
Hex Pairs Calculation: 0.04 seconds
Graph Contains 210 unique hexagons
Average Execution Time: 0.000113 seconds per trip


In [ ]:
def calculate_path_distance_rx(G_rx, path_rx):
    if len(path_rx) < 2:
        return 0.0

    total_distance = 0.0
    for i in range(len(path_rx) - 1):
        u, v = path_rx[i], path_rx[i + 1]
        edge_weight = None
        for edge in G_rx.edges(u):
            if edge[1] == v:
                edge_weight = float(edge[2])
                break
        if edge_weight is None:
            raise ValueError(f"No edge between {u} and {v} in RustworkX")
        total_distance += edge_weight

    return total_distance

def find_and_measure_path(G_rx, origin_rx, destination_rx):
    path_rx = rx.dijkstra_shortest_paths(G_rx, origin_rx, target=destination_rx, weight_fn=lambda x: float(x))

    if destination_rx not in path_rx:
        return None, 0.0

    path_nodes = path_rx[destination_rx]

    distance = calculate_path_distance_rx(G_rx, path_nodes)
    return path_nodes, distance

In [60]:
find_shortest_path(np.random.choice(source_test_nodes), np.random.choice(target_test_nodes))

[229519059,
 229464616,
 229464634,
 229420895,
 229440345,
 229351847,
 229136565,
 229464554,
 229470578,
 3024477727,
 229459627,
 229459542,
 229470583,
 229470587,
 229470592,
 229470600,
 229470607,
 229417891,
 229417887,
 229417886,
 229417883,
 229308652]

In [55]:
def calculate_path_distance(G, path_nx):
    if len(path_nx) < 2:
        return 0.0

    total_distance = 0.0
    for i in range(len(path_nx) - 1):
        u, v = path_nx[i], path_nx[i + 1]
        
        if G.has_edge(u, v):
            # Multi-edge safe approach
            edge_data = G.get_edge_data(u, v)
            if isinstance(edge_data, dict):
                first_edge_key = next(iter(edge_data))
                total_distance += edge_data[first_edge_key]['length']
            else:
                total_distance += edge_data['length']
        
        elif G.has_edge(v, u):
            edge_data = G.get_edge_data(v, u)
            if isinstance(edge_data, dict):
                first_edge_key = next(iter(edge_data))
                total_distance += edge_data[first_edge_key]['length']
            else:
                total_distance += edge_data['length']
        
        else:
            raise ValueError(f"No edge between {u} and {v}")

    return total_distance


In [56]:
test_path = find_shortest_path(np.random.choice(source_test_nodes), np.random.choice(target_test_nodes))

In [57]:
calculate_path_distance(G, test_path)

ValueError: No edge between 229150314 and 229229212

In [ ]:
def get_shortest_path_length(h3_source, h3_target):
    if h3_source == h3_target:
        return 0

    if (h3_source, h3_target) in hex_pairs:
        return hex_pairs[(h3_source, h3_target)]

    source_node = h3_to_anchor_node.get(h3_source)
    target_node = h3_to_anchor_node.get(h3_target)

    if source_node is None or target_node is None:
        raise ValueError("One or both hexes have no representative node")

    source_idx = nx_node_to_rx(source_node)
    target_idx = nx_node_to_rx(target_node)

    path_data = rx.dijkstra_shortest_paths(
        G_rx, source_idx, lambda edge: float(edge)
    )

    distances = path_data["distances"]
    length = distances.get(target_idx, float('inf'))

    return length

In [19]:
# Source: 88446dbb45fffff
# Target: 88446d8eddfffff

In [20]:
get_shortest_path_length('88446dbb45fffff', '88446d8eddfffff')

4625.518999999999

In [29]:
# Function to compute shortest paths between hexagons
def compute_hex_paths(G_rx, hex_to_nodes, source_hex, target_hex):
    """
    Compute shortest paths between all nodes in source_hex to all nodes in target_hex
    """
    paths = {}
    source_nodes = hex_to_nodes[source_hex]
    target_nodes = hex_to_nodes[target_hex]
    
    for source in source_nodes:
        paths[source] = {}
        # Using NetworkX's single_source_dijkstra for efficiency
        lengths, paths_dict = nx.single_source_dijkstra(G, source, weight='length')
        for target in target_nodes:
            if target in lengths:
                paths[source][target] = {
                    'distance': lengths[target],
                    'path': paths_dict[target]
                }
    
    return paths

# Example usage for pre-computing paths between hexagons
def precompute_hex_paths(G, hex_to_nodes):
    """
    Pre-compute shortest paths between all hexagon pairs
    """
    hex_paths = {}
    hexagons = list(hex_to_nodes.keys())
    
    start = timer()
    total_pairs = len(hexagons) * (len(hexagons) - 1) // 2
    computed = 0
    
    for i, hex1 in enumerate(hexagons):
        hex_paths[hex1] = {}
        for hex2 in hexagons[i+1:]:
            hex_paths[hex1][hex2] = compute_hex_paths(G, hex_to_nodes, hex1, hex2)
            computed += 1
            if computed % 10 == 0:  # Progress update every 10 pairs
                print(f"Computed {computed}/{total_pairs} hexagon pairs")
    
    end = timer()
    timetable['path_computation_time'] = end - start
    print(f"Path Computation Time: {end-start:.2f} seconds")
    
    return hex_paths

In [40]:
node_mapping

array([(229134561, -96.3912929, 30.144031 , '88446d8eddfffff',    0),
       (229134590, -96.389991 , 30.144892 , '88446d8eddfffff',    1),
       (229135488, -96.3705723, 30.1715121, '88446d8eb9fffff',    2), ...,
       (11201114401, -96.401673 , 30.1820824, '88446dbb45fffff', 1151),
       (11603136723, -96.3869864, 30.1622537, '88446d8e9dfffff', 1152),
       (11603136727, -96.3874119, 30.1634256, '88446d8e9dfffff', 1153)],
      dtype=[('nx_id', 'O'), ('lon', '<f8'), ('lat', '<f8'), ('hex_id', 'O'), ('rx_idx', '<i8')])

In [41]:
create_hex_pivot(node_mapping)

array([('88446d8535fffff', array([229342684, 229342739, 229409370, 229409378, 229435069, 229504061],
             dtype=object), array([ 82,  83,  91,  92, 347, 779]))                                  ,
       ('88446d8537fffff', array([229416726, 229434330, 229460186, 229460203, 229477721, 229497781,
              6887422915, 6887422916], dtype=object), array([ 125,  346,  521,  522,  641,  740, 1062, 1063])),
       ('88446d85a1fffff', array([229418410, 229421815, 229421820, 229421824, 229424129, 229425047,
              229425052, 229425054, 229425055, 229427563, 229427576, 229427581,
              229433231, 229433235, 229449209, 229454914, 229460327, 229476451,
              229476455, 229487082, 229508145, 229508149, 229508153, 229508168,
              229513611, 1152969720, 1153132503, 1153132507, 3025722594,
              3025771765, 6012957878, 6012957879, 6012957880, 9976292889,
              9976292894, 10649482337, 10649482373], dtype=object), array([ 156,  179,  180,  181,

In [10]:
# Step 5: Save to Disk
start = timer()
with open('precomputed_paths_nx.pkl', 'wb') as f:
    pickle.dump(precomputed_paths_nx, f)
#with open('precomputed_paths_rx.pkl', 'wb') as f:
#    pickle.dump(precomputed_paths_rx, f)
end = timer()
save_time = end - start
timetable['save_time'] = save_time
print(f"Saving Precomputed Paths: {save_time:.2f} seconds")

Saving Precomputed Paths: 0.00 seconds


In [ ]:
# Summary Table
print("\nExecution Time Summary (%d nodes):" % len(G.nodes))
print(f"{'Step':<45}{'Time (s)'}")
print(f"{'-'*55}")
print(f"{'Download Street Network':<45}{timetable['download_time']  :.2f}")
print(f"{'Node-to-H3 Mapping':<45}{timetable['mapping_time']:.2f}")
print(f"{'Shortest Path Precomputation w/ NetworkX':<45}{timetable['precomputation_time_nx']:.2f}")
#print(f"{'Shortest Path Precomputation w/ Rustworkx':<45}{timetable['precomputation_time_rx']:.2f}")
print(f"{'Saving Precomputed Paths':<45}{timetable['save_time']:.2f}")

NameError: name 'G' is not defined

In [84]:
precomputed_paths_rx

{}

In [67]:
precomputed_paths_nx

{('892ab3ad533ffff', '892ab3ae317ffff'): (35,
  [229937272,
   230306670,
   230365662,
   230510482,
   230210551,
   230210564,
   230046113,
   230046084,
   229954902,
   1765710678,
   307612490,
   230001987,
   7243795200,
   7243795244,
   230345843,
   538765023,
   538765231,
   538764687,
   230345667,
   229945765,
   229945786,
   229940182,
   230053211,
   230053185,
   230053150,
   230053131,
   230039544,
   230102287,
   230079020,
   230102312,
   230102325,
   230516513,
   230119679,
   230119718,
   230119741,
   229937390]),
 ('892ab3ae317ffff', '892ab3ad533ffff'): (35,
  [229937390,
   230119741,
   230119718,
   230119679,
   230516513,
   230102325,
   230102312,
   230079020,
   230102287,
   230039544,
   230053131,
   230053150,
   230053185,
   230053211,
   229940182,
   229945786,
   229945765,
   230345667,
   538764687,
   538765231,
   538765023,
   230345843,
   7243795244,
   7243795200,
   230001987,
   307612490,
   1765710678,
   229954902,
   2

In [85]:
source_idx = nx_to_rx_map[nodes[0]]
target_idx = nx_to_rx_map[nodes[1]]
path = rx.dijkstra_shortest_paths(G_rx, source_idx, target_idx)
print(path)

PathMapping{2: [0, 4030, 4489, 5528, 3069, 3070, 847, 846, 84, 6131, 6026, 306, 6648, 6650, 4341, 6033, 6034, 6030, 4340, 46, 47, 21, 987, 986, 985, 984, 727, 1931, 1498, 1932, 1933, 5568, 2231, 2232, 2233, 2]}


In [11]:
print(nx.__version__)

3.4.2
